# Juptyer Notebook - The Outliers

## Authors

Siham Hussein: shusse6@uic.edu, seehamrun@

Fatima Qarni: fqarni2@uic.edu, qarni@

Zaynab Almoujahed: zalmou2@uic.edu, zalmoujahed@ 

Amit Panthi: apanth2@uic.edu, apanth2@

David Qiao: dqiao4@uic.edu, chowsterr@


## Project Overview

The goal of the project is TESTING

## Data Sources 

We used the spotify API to gather information about XXX playlists from XXX different eras. The data about each playlist can be found in the files XXXXXX

## Data Overview 

TODO: Describe the datatype and the content in each of the data files